In [ ]:
import arcpy
arcpy.env.workspace = r"C:\Users\syu\Documents\ArcGIS\Projects\network_1\Default.gdb"

input_lines = "Streets_flagler"
point_layer = "endpoint"
distance_table = "near_distance_table"
output_table = "unconnected_segments"

In [ ]:
# get endpoints
arcpy.FeatureVerticesToPoints_management(input_lines, point_layer, "BOTH_ENDS")
print(f"Generated{point_layer} successfully")

points = {}
point_to_segment ={}

with arcpy.da.SearchCursor(point_layer, ["OBJECTID", "SEGMT_ID", "T_ZLev", "F_ZLev", "SHAPE@XY", "ORIG_FID"]) as cursor:    
    for row in cursor:
        obj_id = row[0]
        seg_id = row[1] if row[1] is not None else "Unknown_" + str(row[0])
        t_z = row[2] if row[2] is not None else 0
        f_z = row[3] if row[3] is not None else 0
        xy = row[4]
        orig_fid = row[5] 
        points[obj_id] = (t_z, f_z, xy)
        point_to_segment[obj_id] = (seg_id, orig_fid)


In [ ]:
# create the distance table 
arcpy.analysis.GenerateNearTable(
    in_features=point_layer,
    near_features=point_layer,
    out_table=distance_table,
    search_radius=1,
    location="NO_LOCATION",
    angle="NO_ANGLE",
    closest="ALL",
    closest_count=0,
    method="PLANAR",  
    distance_unit="METERS"
)

print(f"Generated {distance_table} successfully.")


In [ ]:
# create the output table
arcpy.management.CreateTable(arcpy.env.workspace, output_table)
fields = [
    ("SEGMT_ID_1", "TEXT"),
    ("ORIG_FID_1", "LONG"),
    ("SEGMT_ID_2","TEXT"),
    ("ORIG_FID_2","LONG"),
    ("NEAR_DISTANCE","DOUBLE")
]

arcpy.management.AddFields(output_table, fields)

print (f"Created {output_table}successfully.")


In [ ]:

# filter the distance table and put unconneted points to unconnected_segments
unconnected_pairs = []
with arcpy.da.SearchCursor(distance_table, ["IN_FID", "NEAR_FID", "NEAR_DIST"]) as cursor:
    for row in cursor:
        in_fid = row[0]
        near_fid = row[1]
        near_dist = row[2]
        
        if 0 < near_dist < 1:
            seg1_info = point_to_segment.get(in_fid)
            seg2_info = point_to_segment.get(near_fid)
            
            if seg1_info[1] != seg2_info[1]:
                seg1 = points.get(in_fid)
                seg2 = points.get(near_fid)

                if seg1 and seg2:
                    seg1_t_z, seg1_f_z, _ = seg1
                    seg2_t_z, seg2_f_z, _ = seg2

                    if (seg1_t_z == seg2_t_z) or (seg1_f_z == seg2_f_z) or (seg1_t_z == seg2_f_z) or (seg1_f_z == seg2_t_z):
                        unconnected_pairs.append((
                            seg1_info[0], seg1_info[1],  # SEGMT_ID_1, ORIG_FID_1
                            seg2_info[0], seg2_info[1],  # SEGMT_ID_2, ORIG_FID_2
                            near_dist                    # NEAR_DISTANCE
                        ))
                        print(f"Possible unconnected segments: {seg1_info[0]} and {seg2_info[0]} "
                              f"(Distance: {near_dist:.2f}m, ORIG_FID_1: {seg1_info[1]}, ORIG_FID_2: {seg2_info[1]})")


if unconnected_pairs:
    with arcpy.da.InsertCursor(output_table, ["SEGMT_ID_1", "ORIG_FID_1", "SEGMT_ID_2", "ORIG_FID_2", "NEAR_DISTANCE"]) as cursor:
        for seg1_id, orig_fid1, seg2_id, orig_fid2, dist in unconnected_pairs:
            cursor.insertRow([seg1_id, orig_fid1, seg2_id, orig_fid2, dist])
    print(f"Inserted {len(unconnected_pairs)} records into {output_table}.")
else:
    print("No unconnected segments found.")